<a href="https://colab.research.google.com/github/adelrio89/data_science_for_public_policy/blob/master/Recolecci%C3%B3n_de_Datos_Trabajo_Final_Curso_Aproximaci%C3%B3n_a_las_Pol%C3%ADticas_P%C3%BAblicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Final - Análisis Datos Legislativos Cámara de Diputadxs de Chile
### Recolección de Datos
Andrés Del Río - Felipe Bravo <br>
Diplomado en Ciencia de Datos para Políticas Públicas <br>
Universidad Católica de Chile - Agosto 2021 <br>



---

A continuación el detalle de cómo se obtuvieron los datos y se generaron las bases para los análisis.



#### 1. Obtención de Datos

##### Librerías

In [ ]:
# Consumo de Webservices de OpenData Cámara de Diputadas y Diputados
# Librarías
if (!require("httr")) install.packages("httr") ##herramienta para trabajar URL o HTTP
if (!require("XML")) install.packages("XML")   ##herramienta para trabajar archivos XML
if (!require("xml2")) install.packages("xml2") ##herramienta para trabajar archivos XML
if (!require("tictoc")) install.packages("tictoc") ##medición de tiempos
if (!require("rvest")) install.packages("rvest") ##Permite extraer y manipular datos de una página web
if (!require("openxlsx")) install.packages("openxlsx") ##herramienta para trabajar archivos xlsx
if (!require("tidyverse")) install.packages("tidyverse") ##Conjunto de paquetes para ciencia de datos

library(httr)
library(XML)
library(xml2)
library(tictoc)
library(rvest)
library(openxlsx)
library(tidyverse)

Loading required package: httr

Loading required package: XML

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘XML’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: xml2

Loading required package: tictoc

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘tictoc’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: rvest

Loading required package: openxlsx

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘openxlsx’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: tidyverse

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had st

##### 1.1 Webscraping de Leyes Promulgadas - Cámara de Diputadas y Diputados de Chile

In [ ]:
# descarga lista de leyes promulgadas a la fecha
url <- "https://www.camara.cl/legislacion/ProyectosDeLey/leyes_promulgadas.aspx"
url_html <- read_html(url)
tbl_leyes <- url_html %>% html_table()
# extrae tibble de la lista retornada
tbl_leyes <- tbl_leyes[[1]]
# elimina última columna de la tabla (no tiene datos)
tbl_leyes <- tbl_leyes[seq(1:(length(tbl_leyes)-1))]
# modifica nombre de las columnas
colnames(tbl_leyes) <- c('nrproyecto','dtingreso','nmtipo','nmresumen','nrley','dtpublicacion')
openxlsx::write.xlsx(tbl_leyes,"leyespromulgadas.xlsx")

##### 1.2 Consumo de API Datos de Proyectos Legislativos y Votaciones - Cámara de Diputadas y Diputados de Chile

In [ ]:
# Consumo de webservice Legislativo - Método retornarVotacionesXProyectoLey
# Función que retorna las votaciones de un proyecto (número de boletín)
votacionesProyecto <- function(nrBoletin){
  
  #print(paste('Iniciando:',nrBoletin))
  #nrBoletin = '14260-13'
  #nrBoletin = '12091-03'
  headerFields <- c("Content-Type" = "text/xml; charset=utf-8")
  
  body1 <- paste0('<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Body>
    <retornarVotacionesXProyectoLey xmlns="http://opendata.camara.cl/camaradiputados/v1">
      <prmNumeroBoletin>',nrBoletin,'</prmNumeroBoletin>
    </retornarVotacionesXProyectoLey>
  </soap:Body>
</soap:Envelope>')
  
  # Request
  response1 <- httr::POST(
    url = "http://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx",
    body = body1,
    add_headers(headerFields)
  )
  
  # Se extrae la respuesta en formato XML entregada por el Webservice
  response1 <- content(response1)
  
  #test1 <- xmlTreeParse(response1)[["doc"]]
  
  # Listar paths de xml
  paths <- response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_path()
  
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_name()
  
  response1 <- response1 %>% xml_ns_strip()
  
  idproyecto <- xml_text( xml_find_first( response1, "//Id" ))
  nrboletin <- xml_text( xml_find_first( response1, "//NumeroBoletin" ))
  nmproyecto <- xml_text( xml_find_first( response1, "//Nombre" ))
  dtingreso <- xml_text( xml_find_first( response1, "//FechaIngreso" ))
  nmtipoproyecto <- xml_text( xml_find_first( response1, "//TipoIniciativa" ))
  nmcamaraorigen <- xml_text( xml_find_first( response1, "//CamaraOrigen" ))
  
  votaciones_node <- xml_children(xml_children(xml_children(xml_children(xml_children(response1))))[[8]])
  
  if(length(votaciones_node)>0){
    idvotacion <- votaciones_node %>% map_chr(. %>% xml_find_first(".//Id") %>% xml_text())
    dtvotacion <- votaciones_node %>% map_chr(. %>% xml_find_first(".//Fecha") %>% xml_text())
    totalsi <- votaciones_node %>% map_chr(. %>% xml_find_first(".//TotalSi") %>% xml_text())
    totalno <- votaciones_node %>% map_chr(. %>% xml_find_first(".//TotalNo") %>% xml_text())
    totalabtencion <- votaciones_node %>% map_chr(. %>% xml_find_first(".//TotalAbstencion") %>% xml_text())
    totaldispensados <- votaciones_node %>% map_chr(. %>% xml_find_first(".//TotalDispensado") %>% xml_text())
    nmresultado <- votaciones_node %>% map_chr(. %>% xml_find_first(".//Resultado") %>% xml_text())
    nmtiporesu <- votaciones_node %>% map_chr(. %>% xml_find_first(".//Tipo") %>% xml_text())
    nmtipovota <- votaciones_node %>% map_chr(. %>% xml_find_first(".//TipoVotacionProyectoLey") %>% xml_text())
    nmarticulo <- votaciones_node %>% map_chr(. %>% xml_find_first(".//Articulo") %>% xml_text())
    nmtramcons <- votaciones_node %>% map_chr(. %>% xml_find_first(".//TramiteConstitucional") %>% xml_text())
    nmtramregl <- votaciones_node %>% map_chr(. %>% xml_find_first(".//TramiteReglamentario") %>% xml_text())
    
    dfvotaciones <- data.frame(
      idvotacion, 
      dtvotacion, 
      totalsi,
      totalno,
      totalabtencion,
      totaldispensados,
      nmresultado,
      nmtiporesu,
      nmtipovota,
      nmarticulo,
      nmtramcons,
      nmtramregl
    )
  } else{
    return(data.frame(list()))
  }
  
  dfvotaciones <- cbind(
    idproyecto,
    nrboletin,
    nmproyecto,
    dtingreso,
    nmtipoproyecto,
    nmcamaraorigen,
    dfvotaciones)
  
  if(nmtipoproyecto=='Mocion'){
    autores_node <- xml_children(xml_children(xml_children(xml_children(xml_children(response1))))[[7]])
    if(length(autores_node)>0){
      idautor <- autores_node %>% map_chr(. %>% xml_find_first(".//Id") %>% xml_text())
      nmautor <- autores_node %>% map_chr(. %>% xml_find_first(".//Nombre") %>% xml_text())
      apellidop <- autores_node %>% map_chr(. %>% xml_find_first(".//ApellidoPaterno") %>% xml_text())
      apellidom <- autores_node %>% map_chr(. %>% xml_find_first(".//ApellidoMaterno") %>% xml_text())
      autor <- c(nmautor, apellidop, apellidom)
      idautores <- paste(idautor, collapse = ',')
      nmautores <- paste(autor, collapse = ',')
      dfvotaciones <- cbind(
        dfvotaciones,
        idautores,
        nmautores
      )
    }
  }else if(nmtipoproyecto=='Mensaje'){
    ministerios_node <- xml_children(xml_children(xml_children(xml_children(xml_children(response1))))[[7]])
    if(length(ministerios_node)>0){
      idministerio <- ministerios_node %>% map_chr(. %>% xml_find_first(".//Id") %>% xml_text())
      nmministerio <- ministerios_node %>% map_chr(. %>% xml_find_first(".//Nombre") %>% xml_text())
      idautores <- paste(idministerio, collapse = ',')
      nmautores <- paste(nmministerio, collapse = ',')
      
      dfvotaciones <- cbind(
        dfvotaciones,
        idautores,
        nmautores
      )
    }
  }
  
  materias_node <- xml_children(xml_children(xml_children(xml_children(xml_children(response1))))[[9]])
  
  if(length(materias_node)>0){
    idmateria <- materias_node %>% map_chr(. %>% xml_find_first(".//Id") %>% xml_text())
    nmmateria <- materias_node %>% map_chr(. %>% xml_find_first(".//Nombre") %>% xml_text())
    idmaterias <- paste(idmateria, collapse = ',')
    nmmaterias <- paste(nmmateria, collapse = ',')
    dfvotaciones <- cbind(
      dfvotaciones,
      idmaterias,
      nmmaterias
    )
  }
  #print(paste('Finalizando:',nrBoletin))
  #openxlsx::write.xlsx(dfvotaciones,"votaciones.xlsx")
  return(dfvotaciones)
}

In [ ]:
# Consumo de webservice Legislativo - Método retornarVotacionDetalle
# Función que retorna los votos de lxs diputadxs en una votación
votacion <- function(idVotacion){
  
  #idVotacion = 1847
  headerFields <- c("Content-Type" = "text/xml; charset=utf-8")
  
  body1 <- paste0('<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:v1="http://opendata.camara.cl/camaradiputados/v1">
                   <soapenv:Header/>
                   <soapenv:Body>
                   <v1:retornarVotacionDetalle>
                   <v1:prmVotacionId>', idVotacion, '</v1:prmVotacionId>
                   </v1:retornarVotacionDetalle>
                   </soapenv:Body>
                   </soapenv:Envelope>')
  
  # Request
  response1 <- httr::POST(
    url = "http://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx",
    body = body1,
    add_headers(headerFields)
  )
  
  # Se extrae la respuesta en formato XML entregada por el Webservice
  response1 <- content(response1)
  
  nodes <- xml_children(xml_children(xml_children(response1)))
  # Listar paths de xml
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_path()
  
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_name()
  if(length(nodes)>0){
    response1 <- response1 %>% xml_ns_strip()
    
    general_node <- xml_children(nodes)
    
    idvotacion <- general_node %>% map_chr(. %>% xml_find_first("//Id") %>% xml_text()) %>% unique()
    descripcion <- general_node %>% map_chr(. %>% xml_find_first("//Descripcion") %>% xml_text()) %>% unique()
    dtvotacion <- general_node %>% map_chr(. %>% xml_find_first("//Fecha") %>% xml_text()) %>% unique()
    totalsi <- general_node %>% map_chr(. %>% xml_find_first("//TotalSi") %>% xml_text()) %>% unique()
    totalno <- general_node %>% map_chr(. %>% xml_find_first("//TotalNo") %>% xml_text()) %>% unique()
    totalabstencion <- general_node %>% map_chr(. %>% xml_find_first("//TotalAbstencion") %>% xml_text()) %>% unique()
    totaldispensado <- general_node %>% map_chr(. %>% xml_find_first("//TotalDispensado") %>% xml_text()) %>% unique()
    quorum <- general_node %>% map_chr(. %>% xml_find_first("//Quorum") %>% xml_text()) %>% unique()
    resultado <- general_node %>% map_chr(. %>% xml_find_first("//Resultado") %>% xml_text()) %>% unique()
    tipo <- general_node %>% map_chr(. %>% xml_find_first("//Tipo") %>% xml_text()) %>% unique()
    
    votos_node <- NA
    if(is.na(resultado[[1]])){
      tryCatch(votos_node <- xml_children(xml_children(xml_children(xml_children(xml_children(response1))))[[10]]),
               error = function(e){print(idVotacion)})
    }else{
      tryCatch(votos_node <- xml_children(xml_children(xml_children(xml_children(xml_children(response1))))[[11]]),
               error = function(e){print(idVotacion)})
    }
    
    if(length(votos_node)==0){
      iddiputado <- NA
      nmdiputado <- NA
      nmapellidop <- NA
      nmapellidom <- NA
      nmvoto <- NA
      
      dfvotacion <- data.frame(
        idvotacion,
        descripcion,
        dtvotacion,
        totalsi,
        totalno,
        totalabstencion,
        totaldispensado,
        quorum,
        resultado,
        tipo
      )
      
      dfvotacion <- cbind(
        iddiputado,
        nmdiputado,
        nmapellidop,
        nmapellidom,
        nmvoto,
        dfvotacion
      )
      
    } else{
      
      iddiputado <- votos_node %>% map_chr(. %>% xml_find_first(".//Id") %>% xml_text())
      nmdiputado <- votos_node %>% map_chr(. %>% xml_find_first(".//Nombre") %>% xml_text())
      nmapellidop <- votos_node %>% map_chr(. %>% xml_find_first(".//ApellidoPaterno") %>% xml_text())
      nmapellidom <- votos_node %>% map_chr(. %>% xml_find_first(".//ApellidoMaterno") %>% xml_text())
      nmvoto <- votos_node %>% map_chr(. %>% xml_find_first(".//OpcionVoto") %>% xml_text())
      
      tryCatch(
        dfvotacion <- data.frame(
          iddiputado,
          nmdiputado,
          nmapellidop,
          nmapellidom,
          nmvoto
        ),
        error = function(e){print(idVotacion)})
      
      dfvotacion <- cbind(
        idvotacion,
        descripcion,
        dtvotacion,
        totalsi,
        totalno,
        totalabstencion,
        totaldispensado,
        quorum,
        resultado,
        tipo,
        dfvotacion
      )
    }
    #openxlsx::write.xlsx(dfvotacion,"votacion.xlsx")
    return(dfvotacion)
  } else
    return(data.frame(list()))
}


In [ ]:
# Consumo de webservice Legislativo - Método retornarPeriodosLegislativos
# Función que retorna lista de periodos y legislaturas
periodos <- function(){
  headerFields <- c("Content-Type" = "text/xml; charset=utf-8")
  
  body1 <- '<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:v1="http://opendata.camara.cl/camaradiputados/v1">
   <soapenv:Header/>
   <soapenv:Body>
      <v1:retornarPeriodosLegislativos/>
   </soapenv:Body>
</soapenv:Envelope>'
  
  # Request
  response1 <- httr::POST(
    url = "http://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx",
    body = body1,
    add_headers(headerFields)
  )
  
  # Se extrae la respuesta en formato XML entregada por el Webservice
  response1 <- content(response1)
  
  # Listar paths de xml
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_path()
  
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_name()
  
  response1 <- response1 %>% xml_ns_strip()
  
  
  nrperiodos <- xml_length(xml_child(xml_child(xml_child(response1, 1),1),1))
  
  dflegislaturas <- data.frame(list())
  
  for(i in 1:nrperiodos){
    
    idperiodo <- xml_text(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"Id"))
    nmperiodo <- xml_text(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"Nombre"))
    dtinicio <- xml_text(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"FechaInicio"))
    dttermino <- xml_text(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"FechaTermino"))
    
    # valida si hay legislaturas en el periodo.
    legislaturas <- xml_children(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"Legislaturas"))
    nrlegis <- length(legislaturas)
    if(nrlegis > 0){
      idlegislatura <- xml_text(xml_find_all(xml_children(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"Legislaturas")),"Id"))
      nrlegislatura <- xml_text(xml_find_all(xml_children(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"Legislaturas")),"Numero"))
      dtiniclegi <- xml_text(xml_find_all(xml_children(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"Legislaturas")),"FechaInicio"))
      dttermlegi <- xml_text(xml_find_all(xml_children(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"Legislaturas")),"FechaTermino"))
      nmtipolegi <- xml_text(xml_find_all(xml_children(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"Legislaturas")),"Tipo"))
    } else{
      idlegislatura <- NA
      nrlegislatura <- NA
      dtiniclegi <- NA
      dttermlegi <- NA
      nmtipolegi <- NA
    }
    
    dflegisperiodo <- data.frame(
      idlegislatura,
      nrlegislatura,
      dtiniclegi,
      dttermlegi,
      nmtipolegi
    )
    
    dflegisperiodo <- cbind(
      idperiodo,
      nmperiodo,
      dtinicio,
      dttermino,
      dflegisperiodo
    )
    
    dflegislaturas <- rbind(
      dflegislaturas,
      dflegisperiodo
    )
    
  }
  
  #openxlsx::write.xlsx(dflegislaturas,"legislaturas.xlsx")
  
  return(dflegislaturas)
}


In [ ]:
# Consumo de webservice Legislativo - Método retornarMaterias
# Función que retorna lista de posibles materias en un proyecto
materias <- function(){

  headerFields <- c("Content-Type" = "text/xml; charset=utf-8")
  
  body1 <- '<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:v1="http://opendata.camara.cl/camaradiputados/v1">
   <soapenv:Header/>
   <soapenv:Body>
      <v1:retornarMaterias/>
   </soapenv:Body>
</soapenv:Envelope>'
  
  # Request
  response1 <- httr::POST(
    url = "http://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx",
    body = body1,
    add_headers(headerFields)
  )
  
  # Se extrae la respuesta en formato XML entregada por el Webservice
  response1 <- content(response1)
  
  # Listar paths de xml
  #paths <- response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_path()
  
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_name()
  
  response1 <- response1 %>% xml_ns_strip()
  
  idmateria <- xml_text( xml_find_all( response1, "//Materia/Id" ))
  nmmateria <- xml_text( xml_find_all( response1, "//Materia/Nombre" ))
  
  dfmaterias <- data.frame(
    idmateria,
    nmmateria
  )
  
  #openxlsx::write.xlsx(dfmaterias,"materias.xlsx")
  return(dfmaterias)
}

In [ ]:
# Consumo de webservice Legislativo - Método retornarVotacionesXAnno
# Función que retorna las votaciones de proyectos en un año
votacionesAnio <- function(nranio){
  
  #nranio = 2020
  headerFields <- c("Content-Type" = "text/xml; charset=utf-8")
  
  body1 <- paste0('<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:v1="http://opendata.camara.cl/camaradiputados/v1">
   <soapenv:Header/>
   <soapenv:Body>
      <v1:retornarVotacionesXAnno>
         <v1:prmAnno>', nranio ,'</v1:prmAnno>
      </v1:retornarVotacionesXAnno>
   </soapenv:Body>
</soapenv:Envelope>')
  
  # Request
  response1 <- httr::POST(
    url = "http://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx",
    body = body1,
    add_headers(headerFields)
  )
  
  # Se extrae la respuesta en formato XML entregada por el Webservice
  response1 <- content(response1)
  
  # Listar paths de xml
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_path()
  
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_name()
  
  response1 <- response1 %>% xml_ns_strip()
  
  main_node <- xml_children(xml_children(xml_children(xml_children(response1))))
  
  idvotacion <- main_node %>% map_chr(. %>% xml_find_first(".//Id") %>% xml_text())
  descripcion <- main_node %>% map_chr(. %>% xml_find_first(".//Descripcion") %>% xml_text())
  dtvotacion <- main_node %>% map_chr(. %>% xml_find_first(".//Fecha") %>% xml_text())
  totalsi <- main_node %>% map_chr(. %>% xml_find_first(".//TotalSi") %>% xml_text())
  totalno <- main_node %>% map_chr(. %>% xml_find_first(".//TotalNo") %>% xml_text())
  totalabstencion <- main_node %>% map_chr(. %>% xml_find_first(".//TotalAbstencion") %>% xml_text())
  totaldispensado <- main_node %>% map_chr(. %>% xml_find_first(".//TotalDispensado") %>% xml_text())
  quorum <- main_node %>% map_chr(. %>% xml_find_first(".//Quorum") %>% xml_text())
  resultado <- main_node %>% map_chr(. %>% xml_find_first(".//Resultado") %>% xml_text())
  tipo <- main_node %>% map_chr(. %>% xml_find_first(".//Tipo") %>% xml_text())
  
  dfvotacionxanio <- data.frame(
    idvotacion,
    descripcion,
    dtvotacion,
    totalsi,
    totalno,
    totalabstencion,
    totaldispensado,
    quorum,
    resultado,
    tipo
  )
  
  #openxlsx::write.xlsx(dfvotacionxanio,"votacionxanio.xlsx")
  return(dfvotacionxanio)
}

In [ ]:
# Consumo de webservice Legislativo - Método retornarMensajesXAnno
# Función que retorna los proyectos tipo "Mensajes" de un año
mensajesAnio <- function(nranio){
  
  #nranio = 2020
  headerFields <- c("Content-Type" = "text/xml; charset=utf-8")
  
  body1 <- paste0('<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:v1="http://opendata.camara.cl/camaradiputados/v1">
   <soapenv:Header/>
   <soapenv:Body>
      <v1:retornarMensajesXAnno>
         <v1:prmAnno>', nranio , '</v1:prmAnno>
      </v1:retornarMensajesXAnno>
   </soapenv:Body>
</soapenv:Envelope>')
  
  # Request
  response1 <- httr::POST(
    url = "http://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx",
    body = body1,
    add_headers(headerFields)
  )
  
  # Se extrae la respuesta en formato XML entregada por el Webservice
  response1 <- content(response1)
  
  # Listar paths de xml
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_path()
  
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_name()
  
  response1 <- response1 %>% xml_ns_strip()
  
  main_node <- xml_children(xml_children(xml_children(xml_children(response1))))
  
  idproyecto <- main_node %>% map_chr(. %>% xml_find_first(".//Id") %>% xml_text())
  nrboletin <- main_node %>% map_chr(. %>% xml_find_first(".//NumeroBoletin") %>% xml_text())
  nmproyecto <- main_node %>% map_chr(. %>% xml_find_first(".//Nombre") %>% xml_text())
  dtingreso <- main_node %>% map_chr(. %>% xml_find_first(".//FechaIngreso") %>% xml_text())
  nmtipo <- main_node %>% map_chr(. %>% xml_find_first(".//TipoIniciativa") %>% xml_text())
  nmcamaorig <- main_node %>% map_chr(. %>% xml_find_first(".//CamaraOrigen") %>% xml_text())
  nmadmisible <- main_node %>% map_chr(. %>% xml_find_first(".//Admisible") %>% xml_text())
  
  dfmensajesxanio <- data.frame(
    idproyecto,
    nrboletin,
    nmproyecto,
    dtingreso,
    nmtipo,
    nmcamaorig,
    nmadmisible
  )
  
  #openxlsx::write.xlsx(dfmensajesxanio,"mensajesxanio.xlsx")
  return(dfmensajesxanio)
  
}

In [ ]:
# Consumo de webservice Legislativo - Método retornarMocionesXAnno
# Función que retorna los proyectos tipo "Moción" de un año
mocionesAnio <- function(nranio){
  
  #nranio = 2020
  headerFields <- c("Content-Type" = "text/xml; charset=utf-8")
  
  body1 <- paste0('<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:v1="http://opendata.camara.cl/camaradiputados/v1">
   <soapenv:Header/>
   <soapenv:Body>
      <v1:retornarMocionesXAnno>
         <v1:prmAnno>', nranio ,'</v1:prmAnno>
      </v1:retornarMocionesXAnno>
   </soapenv:Body>
</soapenv:Envelope>')
  
  # Request
  response1 <- httr::POST(
    url = "http://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx",
    body = body1,
    add_headers(headerFields)
  )
  
  # Se extrae la respuesta en formato XML entregada por el Webservice
  response1 <- content(response1)
  
  # Listar paths de xml
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_path()
  
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_name()
  
  response1 <- response1 %>% xml_ns_strip()
  
  main_node <- xml_children(xml_children(xml_children(xml_children(response1))))
  
  idproyecto <- main_node %>% map_chr(. %>% xml_find_first(".//Id") %>% xml_text())
  nrboletin <- main_node %>% map_chr(. %>% xml_find_first(".//NumeroBoletin") %>% xml_text())
  nmproyecto <- main_node %>% map_chr(. %>% xml_find_first(".//Nombre") %>% xml_text())
  dtingreso <- main_node %>% map_chr(. %>% xml_find_first(".//FechaIngreso") %>% xml_text())
  nmtipo <- main_node %>% map_chr(. %>% xml_find_first(".//TipoIniciativa") %>% xml_text())
  nmcamaorig <- main_node %>% map_chr(. %>% xml_find_first(".//CamaraOrigen") %>% xml_text())
  nmadmisible <- main_node %>% map_chr(. %>% xml_find_first(".//Admisible") %>% xml_text())
  
  dfmocionesxanio <- data.frame(
    idproyecto,
    nrboletin,
    nmproyecto,
    dtingreso,
    nmtipo,
    nmcamaorig,
    nmadmisible
  )

  #openxlsx::write.xlsx(dfmocionesxanio,"mocionesxanio.xlsx")
  return(dfmocionesxanio)
  
}

In [ ]:
# Consumo de webservice Legislativo - Método retornarVotacionesXProyectoLey
# Función que retorna los autores/patrocinadores de un proyecto
autoresProyecto <- function(nrBoletin){
  #nrBoletin = '12598-21'
  headerFields <- c("Content-Type" = "text/xml; charset=utf-8")
  
  body1 <- paste0('<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Body>
    <retornarVotacionesXProyectoLey xmlns="http://opendata.camara.cl/camaradiputados/v1">
      <prmNumeroBoletin>',nrBoletin,'</prmNumeroBoletin>
    </retornarVotacionesXProyectoLey>
  </soap:Body>
</soap:Envelope>')
  
  # Request
  response1 <- httr::POST(
    url = "http://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx",
    body = body1,
    add_headers(headerFields)
  )
  
  # Se extrae la respuesta en formato XML entregada por el Webservice
  response1 <- content(response1)
  
  #test1 <- xmlTreeParse(response1)[["doc"]]
  
  # Listar paths de xml
  paths <- response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_path()
  
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_name()
  
  response1 <- response1 %>% xml_ns_strip()
  
  dfautores <- data.frame(list())
  
  idproyecto <- xml_text( xml_find_first( response1, "//Id" ))
  nrboletin <- xml_text( xml_find_first( response1, "//NumeroBoletin" ))
  nmtipoproyecto <- xml_text( xml_find_first( response1, "//TipoIniciativa" ))
  nmcamaraorigen <- xml_text( xml_find_first( response1, "//CamaraOrigen" ))
  
  if(nmtipoproyecto=='Mocion' | nmtipoproyecto=='Moción'){
    autores_node <- xml_children(xml_children(xml_children(xml_children(xml_children(response1))))[[7]])
    if(length(autores_node)>0){
      idautor <- autores_node %>% map_chr(. %>% xml_find_first(".//Id") %>% xml_text())
      nmautor <- autores_node %>% map_chr(. %>% xml_find_first(".//Nombre") %>% xml_text())
      apellidop <- autores_node %>% map_chr(. %>% xml_find_first(".//ApellidoPaterno") %>% xml_text())
      apellidom <- autores_node %>% map_chr(. %>% xml_find_first(".//ApellidoMaterno") %>% xml_text())
      autor <- paste(nmautor, apellidop, apellidom)
    }else{
      idautor <- NA
      autor <- NA 
    }
  }
  if(nmtipoproyecto=='Mensaje'){
    ministerios_node <- xml_children(xml_children(xml_children(xml_children(xml_children(response1))))[[7]])
    if(length(ministerios_node)>0){
      idautor <- ministerios_node %>% map_chr(. %>% xml_find_first(".//Id") %>% xml_text())
      autor <- ministerios_node %>% map_chr(. %>% xml_find_first(".//Nombre") %>% xml_text())
    }else{
      idautor <- NA
      autor <- NA
    }
  }
  
  dfautores <- data.frame(
    idautor,
    autor
  )
  
  dfautores <- cbind(
    idproyecto,
    nrboletin,
    nmcamaraorigen,
    dfautores
  )
  #print(paste('Finalizando:',nrBoletin))
  #openxlsx::write.xlsx(dfautores,"test1.xlsx")
  return(dfautores)
}

In [ ]:
# Consumo de webservice Legislativo - Método retornarVotacionesXProyectoLey
# Función que retorna las materias de un proyecto
materiasProyecto <- function(nrBoletin){
  #nrBoletin = '14260-13'
  #nrBoletin = '12091-03'
  headerFields <- c("Content-Type" = "text/xml; charset=utf-8")
  
  body1 <- paste0('<?xml version="1.0" encoding="utf-8"?>
<soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
  <soap:Body>
    <retornarVotacionesXProyectoLey xmlns="http://opendata.camara.cl/camaradiputados/v1">
      <prmNumeroBoletin>',nrBoletin,'</prmNumeroBoletin>
    </retornarVotacionesXProyectoLey>
  </soap:Body>
</soap:Envelope>')
  
  # Request
  response1 <- httr::POST(
    url = "http://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx",
    body = body1,
    add_headers(headerFields)
  )
  
  # Se extrae la respuesta en formato XML entregada por el Webservice
  response1 <- content(response1)
  
  #test1 <- xmlTreeParse(response1)[["doc"]]
  
  # Listar paths de xml
  paths <- response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_path()
  
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_name()
  
  response1 <- response1 %>% xml_ns_strip()
  
  idproyecto <- xml_text( xml_find_first( response1, "//Id" ))
  nrboletin <- xml_text( xml_find_first( response1, "//NumeroBoletin" ))
  
  materias_node <- xml_children(xml_children(xml_children(xml_children(xml_children(response1))))[[9]])
  
  if(length(materias_node)>0){
    idmateria <- materias_node %>% map_chr(. %>% xml_find_first(".//Id") %>% xml_text())
    nmmateria <- materias_node %>% map_chr(. %>% xml_find_first(".//Nombre") %>% xml_text())
  }else{
    idmateria <- NA
    nmmateria <- NA
  }
  
  dfmateriasproy <- data.frame(
    idmateria,
    nmmateria
  )
  
  dfmateriasproy <- cbind(
    idproyecto,
    nrboletin,
    dfmateriasproy
  )
  #print(paste('Finalizando:',nrBoletin))
  #openxlsx::write.xlsx(dfmateriasproy,"test1.xlsx")
  return(dfmateriasproy)
}

##### 1.3 Consumo de API Datos de Diputadxs - Cámara de Diputadas y Diputados de Chile

In [ ]:
# Consumo de webservice Diputado - Método retornarDiputados
# Función que retorna los diputados (actuales e históricos)
diputados <- function(){
  
  #idVotacion = 1847
  headerFields <- c("Content-Type" = "text/xml; charset=utf-8")
  
  body1 <- '<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:v1="http://opendata.camara.cl/camaradiputados/v1">
  <soapenv:Header/>
  <soapenv:Body>
  <v1:retornarDiputados/>
  </soapenv:Body>
  </soapenv:Envelope>'
  
  # Request
  response1 <- httr::POST(
    url = "http://opendata.camara.cl/camaradiputados/WServices/WSDiputado.asmx",
    body = body1,
    add_headers(headerFields)
  )
  
  # Se extrae la respuesta en formato XML entregada por el Webservice
  response1 <- content(response1)
  
  # Listar paths de xml
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_path()
  
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_name()
  
  response1 <- response1 %>% xml_ns_strip()
  
  general_node <- xml_children(xml_children(xml_children(xml_children(response1))))
  nrdiputados <- length(general_node)
  
  dflistdipu <- data.frame(list())
  
  for(i in 1:nrdiputados){
    
    iddiputado <-  xml_text(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"Id"))
    nmdiputado <- xml_text(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"Nombre"))
    apellidop <- xml_text(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"ApellidoPaterno"))
    apellidom <- xml_text(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"ApellidoMaterno"))
    dtnacimiento <- xml_text(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"FechaNacimiento"))
    nmsexo <- xml_text(xml_child(xml_children(xml_children(xml_children(xml_children(response1))))[[i]],"Sexo"))
    
    militancias <- xml_children(xml_child(general_node[[i]],"Militancias"))
    nrmilit <- length(militancias)
    
    if(nrmilit > 0){
      dtinicio <- militancias %>% xml_find_first(".//FechaInicio") %>% xml_text()
      dttermino <- militancias %>% xml_find_first(".//FechaTermino") %>% xml_text()
      idpartido <- militancias %>% xml_find_first(".//Partido/Id") %>% xml_text()
      nmpartido <- militancias %>% xml_find_first(".//Partido/Nombre") %>% xml_text()
      alias <- militancias %>% xml_find_first(".//Partido/Alias") %>% xml_text()
    } else{
      dtinicio <- NA
      dttermino <- NA
      idpartido <- NA
      nmpartido <- NA
      alias <- NA
    }
    
    dfdiputado <- data.frame(
      dtinicio,
      dttermino,
      idpartido,
      nmpartido,
      alias
    )
    
    dfdiputado <- cbind(
      iddiputado,
      nmdiputado,
      apellidop,
      apellidom,
      dtnacimiento,
      nmsexo,
      dfdiputado
    )
    
    dflistdipu <- rbind(
      dflistdipu,
      dfdiputado
    )
    
  }
  
  
  #openxlsx::write.xlsx(dflistdipu,"diputadxs.xlsx")
  return(dflistdipu)
}

In [ ]:
# Consumo de webservice Diputado - Método retornarDiputadosXPeriodo
# Función que retorna los diputados agrupados por periodo legislativo
diputadosxPeriodo <- function(nrPeriodo){
  
  #nrPeriodo = 8
  headerFields <- c("Content-Type" = "text/xml; charset=utf-8")
  
  body1 <- paste0('<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:v1="http://opendata.camara.cl/camaradiputados/v1">
   <soapenv:Header/>
   <soapenv:Body>
      <v1:retornarDiputadosXPeriodo>
         <v1:prmPeriodoID>', nrPeriodo, '</v1:prmPeriodoID>
      </v1:retornarDiputadosXPeriodo>
   </soapenv:Body>
</soapenv:Envelope>')
  
  # Request
  response1 <- httr::POST(
    url = "http://opendata.camara.cl/camaradiputados/WServices/WSDiputado.asmx",
    body = body1,
    add_headers(headerFields)
  )
  
  # Se extrae la respuesta en formato XML entregada por el Webservice
  response1 <- content(response1)
  
  # Listar paths de xml
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_path()
  
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_name()
  
  response1 <- response1 %>% xml_ns_strip()
  
  general_node <- xml_children(xml_children(xml_children(xml_children(response1))))
  nrdiputados <- length(general_node)
  
  dflistdipu <- data.frame(list())
  
  if(nrdiputados == 0){
    return(data.frame(list()))
  }
  
  for(i in 1:nrdiputados){
    
    dtinicio <- xml_text(xml_child(general_node[[i]],"FechaInicio"))
    dttermino <- xml_text(xml_child(general_node[[i]],"FechaTermino"))
    iddiputado <-xml_text(xml_child(xml_child(general_node[[i]],"Diputado"),"Id"))
    nmdiputado <- xml_text(xml_child(xml_child(general_node[[i]],"Diputado"),"Nombre"))
    apellidop <- xml_text(xml_child(xml_child(general_node[[i]],"Diputado"),"ApellidoPaterno"))
    apellidom <- xml_text(xml_child(xml_child(general_node[[i]],"Diputado"),"ApellidoMaterno"))
    dtnacimiento <- xml_text(xml_child(xml_child(general_node[[i]],"Diputado"),"FechaNacimiento"))
    nmsexo <- xml_text(xml_child(xml_child(general_node[[i]],"Diputado"),"Sexo"))
    
    
    distrito <- xml_children(xml_child(general_node[[i]],"Distrito"))
    
    
    nrelem <- length(distrito)
    
    if(nrelem > 0){
      nrdistrito <- distrito[[1]] %>% xml_text() 
      nrcomunas <- distrito %>% xml_find_all(".//Comuna/Numero") %>% xml_text()
      nmcomunas <- distrito %>% xml_find_all(".//Comuna/Nombre") %>% xml_text()
    } else{
      nrdistrito <- NA
      nrcomunas <- NA
      nmcomunas <- NA
    }
    
    dfdiputado <- data.frame(
      nrcomunas,
      nmcomunas
    )
    
    tryCatch(
      dfdiputado <- cbind(
        nrdistrito,
        dfdiputado
      ), error = function(e){print(iddiputado)})
    
    periodo <- nrPeriodo
    dfdiputado <- cbind(
      periodo,
      dtinicio,
      dttermino,
      iddiputado,
      nmdiputado,
      apellidop,
      apellidom,
      dtnacimiento,
      nmsexo,
      dfdiputado
    )
    
    dflistdipu <- rbind(
      dflistdipu,
      dfdiputado
    )
    
  }
  
  #openxlsx::write.xlsx(dflegislaturas,"legislaturas.xlsx")
  return(dflistdipu)
}

##### 1.4 Consumo de API Datos Genéricos - Cámara de Diputadas y Diputados de Chile

In [ ]:
# Consumo de webservice Común - Método retornarDistritos
# Función que retorna los distritos electorales y sus comunas
distritos <- function(){
  
  headerFields <- c("Content-Type" = "text/xml; charset=utf-8")
  
  body1 <- '<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:v1="http://opendata.camara.cl/camaradiputados/v1">
    <soapenv:Header/>
    <soapenv:Body>
    <v1:retornarDistritos/>
    </soapenv:Body>
    </soapenv:Envelope>'
  
  # Request
  response1 <- httr::POST(
    url = "http://opendata.camara.cl/camaradiputados/WServices/WSComun.asmx",
    body = body1,
    add_headers(headerFields)
  )
  
  # Se extrae la respuesta en formato XML entregada por el Webservice
  response1 <- content(response1)
  
  # Listar paths de xml
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_path()
  
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_name()
  
  response1 <- response1 %>% xml_ns_strip()
  
  general_node <- xml_children(xml_children(xml_children(xml_children(response1))))
  nrdistritos <- length(general_node)
  
  dfdistritos <- data.frame(list())
  
  if(nrdistritos == 0){
    return(dfdistritos)
  }
  
  for(i in 1:nrdistritos){
    
    iddistrito <- xml_text(xml_child(general_node[[i]],"Numero"))
    
    comunas_node <- xml_children(xml_children(general_node[[i]]))
    if(length(comunas_node)>0){
      nrcomuna <- comunas_node %>% xml_find_all(".//Numero") %>% xml_text()
      nmcomuna <- comunas_node %>% xml_find_all(".//Nombre") %>% xml_text()
    } else{
      nrcomuna <- NA
      nmcomuna <- NA
    }
    
    dfcomunas <- data.frame(
      nrcomuna,
      nmcomuna
    )
    
    dfcomunas <- cbind(
      iddistrito,
      dfcomunas
    )
    
    dfdistritos <- rbind(
      dfdistritos,
      dfcomunas
    )
    
  }
  
  
  #openxlsx::write.xlsx(dflegislaturas,"legislaturas.xlsx")
  
  return(dfdistritos)
}

In [ ]:
# Consumo de webservice Común - Método retornarRegiones
# Función que retorna las regiones con sus respectivas provincias y comunas
regiones <- function(){
  
  headerFields <- c("Content-Type" = "text/xml; charset=utf-8")
  
  body1 <- '<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:v1="http://opendata.camara.cl/camaradiputados/v1">
   <soapenv:Header/>
   <soapenv:Body>
      <v1:retornarRegiones/>
   </soapenv:Body>
</soapenv:Envelope>'
  
  # Request
  response1 <- httr::POST(
    url = "http://opendata.camara.cl/camaradiputados/WServices/WSComun.asmx",
    body = body1,
    add_headers(headerFields)
  )
  
  # Se extrae la respuesta en formato XML entregada por el Webservice
  response1 <- content(response1)
  
  # Listar paths de xml
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_path()
  
  #response1 %>% xml_ns_strip() %>% xml_find_all( '//*') %>% xml_name()
  
  response1 <- response1 %>% xml_ns_strip()
  
  general_node <- xml_children(xml_children(xml_children(xml_children(response1))))
  nrregiones <- length(general_node)
  
  dfregiones <- data.frame(list())
  
  if(nrregiones == 0){
    return(dfregiones)
  }
  
  for(i in 1:nrregiones){
    nrregion <- xml_text(xml_child(general_node[[i]],"Numero"))
    romano <- xml_text(xml_child(general_node[[i]],"NumeroRomano"))
    nmregion <- xml_text(xml_child(general_node[[i]],"Nombre"))
    
    provincias_node <- xml_children(xml_children(general_node[[i]]))
    dfcomunas <- data.frame(list())
    
    if(length(provincias_node)>0){
      
      for(j in 1:length(provincias_node)){
        nrprovincia <- xml_child(provincias_node[[j]],"Numero") %>% xml_text()
        nmprovincia <- xml_child(provincias_node[[j]],"Nombre") %>% xml_text()
        comunas_node <- xml_children(xml_child(provincias_node[[j]],"Comunas"))
        cant <- length(comunas_node)
        if(cant>0){
          nrcomuna <- xml_child(comunas_node,"Numero") %>% xml_text()
          nmcomuna <- xml_child(comunas_node,"Nombre") %>% xml_text()    
        }else{
          nrcomuna <- NA
          nmcomuna <- NA
        }
        dfcomunas <- data.frame(
          nrcomuna,
          nmcomuna
        )
        dfcomunas <- cbind(
          nrprovincia,
          nmprovincia,
          dfcomunas
        )
        
        dfcomunas <- cbind(
          nrregion,
          romano,
          nmregion,
          dfcomunas
        )
        
        dfregiones <- rbind(
          dfregiones,
          dfcomunas
        )
        
      }
      
      dfregiones <- rbind(
        dfregiones,
        dfcomunas
      )
      
      
    } else{
      return(dfregiones)
    }
  }
  
  #openxlsx::write.xlsx(dflegislaturas,"legislaturas.xlsx")
  
  dfregiones <- dfregiones %>% unique()
  return(dfregiones)
}

##### 1.5 Datos Vectoriales por Comunas

In [ ]:
fileName <- "http://geonode.meteochile.gob.cl/geoserver/wfs?format_options=charset%3AUTF-8&typename=geonode%3Adivision_comunal_geo_ide_1&outputFormat=SHAPE-ZIP&version=1.0.0&service=WFS&request=GetFeature"
download.file(fileName, destfile = "comunas.zip")
#unzip("comunas.zip",exdir = "./content")

#### 2. Generación de Bases para Análisis

In [ ]:
# Extrae el número de boletín de todas las leyes publicadas
nrboletines <- tbl_leyes$nrproyecto

In [ ]:
# Crea base de autores de leyes promulgadas
tic()
tbl_autores <- map_dfr(nrboletines, autoresProyecto)
openxlsx::write.xlsx(tbl_autores,"autoresleyespromulgadas.xlsx")
toc()

555.713 sec elapsed


In [ ]:
# Crea base de materias de leyes promulgadas
tic()
tbl_materiasproy <- map_dfr(nrboletines, materiasProyecto)
openxlsx::write.xlsx(tbl_materiasproy,"materiasleyespromulgadas.xlsx")
toc()

563.813 sec elapsed


In [ ]:
# Crea base de votaciones de leyes promulgadas - 
tic()
#votacionesProyecto('12091-03')
votaciones_leyes <- map_dfr(nrboletines, votacionesProyecto)
openxlsx::write.xlsx(votaciones_leyes,"votacionesleyespromulgadas.xlsx")
# 10846 votaciones de la lista de proyectos publicados en página web al 30/07/2021
toc()

586.061 sec elapsed


In [ ]:
# Crea base con detalle de votaciones de leyes promulgadas
# Este código tardó casi 1 hora en ejecutarse.
tic()
#votacion(1847)
idvotaciones <- votaciones_leyes %>% select(idvotacion) %>% distinct()
#nrow(idvotaciones)
ids <- as.integer(as.vector(as.character(idvotaciones$idvotacion)))
detallvotos_leyes <- map_dfr(ids, votacion)
#nrow(detallvotos_leyes)
openxlsx::write.xlsx(detallvotos_leyes,"detallevotosleyespromulgadas.xlsx")
toc()

3098.598 sec elapsed


In [ ]:
# Crear base con datos de diputadxs
tic()
tbl_diputados <- diputados()
openxlsx::write.xlsx(tbl_diputados,"diputadxs.xlsx")
toc()

15.393 sec elapsed


In [ ]:
# Crea base con datos de diputadxs agrupadxs por periodos legislativos
tic()
tbl_totalperiod <- map_dfr(c(1,2,3,4,5,6,8,9),diputadosxPeriodo)
openxlsx::write.xlsx(tbl_totalperiod,"diputadxstodosperiodos.xlsx")
toc()

6.555 sec elapsed


In [ ]:
# Crea base con distritos electores
tic()
tbl_distritos <- distritos()
openxlsx::write.xlsx(tbl_distritos,"distritos.xlsx")
toc()

0.321 sec elapsed


In [ ]:
# Crea base con regiones y sus respectivas provincias y comunas
tic()
tbl_regiones <- regiones()
openxlsx::write.xlsx(tbl_regiones,"regiones.xlsx")
toc()

0.514 sec elapsed


In [ ]:
# Crea base con periodos legislativos
tic()
tbl_periodos <- periodos()
openxlsx::write.xlsx(tbl_periodos,"periodos.xlsx")
toc()

0.297 sec elapsed
